<a href="https://colab.research.google.com/github/VtlNmnk/crema/blob/master/02_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Get access to google drive folder.
# This only needs to be done once per notebook.
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/ACE/'  #change dir to your project folder

# Change current directoty to mounted on gdrive
os.chdir(root_path)
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/My Drive/ACE'

In [3]:
import tensorflow
device_name = tensorflow.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14004496103599693379, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12250426452105035657
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13375306243299948867
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18293539097563749402
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [5]:
!ls

 augment_results_collection   output_results
'Colab Notebooks'	      output_results_queen
 crema			      output_results_TPU
 datasets		      pumpp
 example		      __pycache__
 libs			      pyrubberband
 log_ACE		      requirements.txt
 muda			      resources
 output_example_results       rubberband-cli_1.8.1-4_amd64.deb


In [0]:
# Maximum number of samples to draw per streamer
max_samples = 128

# Duration (in seconds) of training patches
duration = 8.0

# Seed for the random number generator
seed = 100

# Number of active training streamers
train_streamers = 1024


# Size of training batches
batch_size = 16
# batch_size = 128

# Rate of pescador stream deactivation
rate = 16

# Maximum number of epochs to train for
epochs = 500

# Number of batches per epoch
epoch_size = 2048
# epoch_size = 12

# Number of epochs without improvement to stop
early_stopping = 50

# Number of epochs before reducing learning rate
reduce_lr = 20

# Path to working directory
# working = os.path.join(root_path, "output_results")
working = "output_results_queen"

# Path to output directory
# output_path = os.path.join(root_path, "output_results")
output_path = "output_results_queen/outputs"

In [0]:
def make_sampler(max_samples, duration, pump, seed):
    '''stochastic training sampler'''
    n_frames = librosa.time_to_frames(duration,
                                      sr=pump['cqt'].sr,
                                      hop_length=pump['cqt'].hop_length)
    return pump.sampler(max_samples, n_frames, random_state=seed)

def val_sampler(max_duration, pump, seed):
    '''validation sampler'''
    n_frames = librosa.time_to_frames(max_duration,
                                      sr=pump['cqt'].sr,
                                      hop_length=pump['cqt'].hop_length)
    return pumpp.sampler.VariableLengthSampler(None, 32, n_frames,
                                               *pump.ops,
                                               random_state=seed)

def data_sampler(fname, sampler):
    '''Generate samples from a specified h5 file'''
    for datum in sampler(custom_utils.load_h5(fname)):
        yield datum

def data_generator(working, tracks, sampler, k, augment=True, rate=8, **kwargs):
    '''Generate a data stream from a collection of tracks and a sampler'''
    seeds = []
    for track in tqdm(tracks,
             desc='Collecting train data'):
        fname = os.path.join(working,
                             os.path.extsep.join([track, 'h5']))
        seeds.append(pescador.Streamer(data_sampler, fname, sampler))

        
    # Send it all to a mux
    return pescador.StochasticMux(seeds, k, rate, **kwargs)

def val_generator(working, tracks, sampler, augment=True):
    '''validation generator, deterministic roundrobin'''
    seeds = []
    for track in tqdm(tracks,
             desc='Collecting val data'):
        fname = os.path.join(working,
                             os.path.extsep.join([track, 'h5']))
        seeds.append(pescador.Streamer(data_sampler, fname, sampler))
       
    # Send it all to a mux
    return pescador.RoundRobinMux(seeds)



def construct_model(pump):
    '''build the model'''
    model_inputs = 'cqt/mag'

    # Build the input layer
    x = pump.layers()[model_inputs]

    # Apply batch normalization
    x_bn = K.layers.BatchNormalization()(x)

    # First convolutional filter: a single 5x5
    conv1 = K.layers.Convolution2D(1, (5, 5),
                                   padding='same',
                                   activation='relu',
                                   data_format='channels_last')(x_bn)

    c1bn = K.layers.BatchNormalization()(conv1)

    # Second convolutional filter: a bank of full-height filters
    conv2 = K.layers.Convolution2D(12*6, (1, int(conv1.shape[2])),
                                   padding='valid', activation='relu',
                                   data_format='channels_last')(c1bn)

    c2bn = K.layers.BatchNormalization()(conv2)

    # Squeeze out the frequency dimension
    squeeze = custom_layers.SqueezeLayer(axis=2)(c2bn)

    # BRNN layer
    rnn1 = K.layers.Bidirectional(K.layers.GRU(128,
                                               return_sequences=True))(squeeze)

    r1bn = K.layers.BatchNormalization()(rnn1)

    rnn2 = K.layers.Bidirectional(K.layers.GRU(128,
                                              return_sequences=True))(r1bn)

    # 1: pitch class predictor
    pc = K.layers.Dense(pump.fields['chord_struct/pitch'].shape[1],
                        activation='sigmoid')

    pc_p = K.layers.TimeDistributed(pc, name='chord_pitch')(rnn2)

    # 2: root predictor
    root = K.layers.Dense(13, activation='softmax')
    root_p = K.layers.TimeDistributed(root, name='chord_root')(rnn2)

    # 3: bass predictor
    bass = K.layers.Dense(13, activation='softmax')
    bass_p = K.layers.TimeDistributed(bass, name='chord_bass')(rnn2)

    # 4: merge layer
    codec = K.layers.concatenate([rnn2, pc_p, root_p, bass_p])

    codecbn = K.layers.BatchNormalization()(codec)

    p0 = K.layers.Dense(len(pump['chord_tag'].vocabulary()),
                        activation='softmax',
                        bias_regularizer=K.regularizers.l2())

    tag = K.layers.TimeDistributed(p0, name='chord_tag')(codecbn)

    model = K.models.Model(x, [tag, pc_p, root_p, bass_p])
    model_outputs = ['chord_tag/chord',
                     'chord_struct/pitch',
                     'chord_struct/root',
                     'chord_struct/bass']

    return model, [model_inputs], model_outputs

def prepare_model(working, max_samples, duration, rate,
          batch_size, epochs, epoch_size,
          early_stopping, reduce_lr, seed, checkpoint_path=None):
    '''
    Parameters
    ----------
    working : str
        directory that contains the experiment data (h5)
    max_samples : int
        Maximum number of samples per streamer
    duration : float
        Duration of training patches
    batch_size : int
        Size of batches
    rate : int
        Poisson rate for pescador
    epochs : int
        Maximum number of epoch
    epoch_size : int
        Number of batches per epoch
    validation_size : int
        Number of validation batches
    early_stopping : int
        Number of epochs before early stopping
    reduce_lr : int
        Number of epochs before reducing learning rate
    seed : int
        Random seed
    '''
    # Load the pump
    with open(os.path.join(working, 'pump.pkl'), 'rb') as fd:
        pump = pickle.load(fd)
    # Build the sampler
    sampler = make_sampler(max_samples, duration, pump, seed)
    # And the validation sampler: cap out at 10 minutes
    # sampler_val = val_sampler(10 * 60, pump, seed)#too slow

    sampler_val = val_sampler(10, pump, seed)#test
    # Build the model
    model, inputs, outputs = construct_model(pump)
    # Load the training data
    idx_train_ = pd.read_json(os.path.join(working, 'index_train.json'))
    # Split the training data into train and validation
    splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                               random_state=seed)
    train, val = next(splitter_tv.split(idx_train_))
    idx_train = idx_train_.iloc[train]
    idx_val = idx_train_.iloc[val]
    gen_train = data_generator(working,
                               idx_train['id'].values, sampler, epoch_size,
                               augment=True,
                               rate=rate,
                               mode='with_replacement',
                               random_state=seed)
    
    gen_train = pescador.maps.keras_tuples(pescador.maps.buffer_stream(gen_train(),
                                                                       batch_size,
                                                                       axis=0),
                                           inputs=inputs,
                                           outputs=outputs)
    gen_val = val_generator(working,
                            idx_val['id'].values, sampler_val,
                            augment=True)
    validation_size = gen_val.n_streams
    gen_val = pescador.maps.keras_tuples(gen_val(),
                                         inputs=inputs,
                                         outputs=outputs)

    loss = {'chord_tag': 'sparse_categorical_crossentropy'}
    metrics = {'chord_tag': 'sparse_categorical_accuracy'}
    loss.update(chord_pitch='binary_crossentropy',
                chord_root='sparse_categorical_crossentropy',
                chord_bass='sparse_categorical_crossentropy')
    # monitor = 'val_chord_tag_sparse_categorical_accuracy'
    monitor = 'val_chord_tag_loss'
    # sgd = K.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    adam = K.optimizers.Adam(lr=0.1e-5)

    model.compile(optimizer=adam, loss=loss, metrics=metrics)
    # model.compile(sgd, loss=loss, metrics=metrics)

    # Store the model
    model_spec = K.utils.serialize_keras_object(model)
    with open(os.path.join(output_path, 'model_spec.pkl'), 'wb') as fd:
        pickle.dump(model_spec, fd)

    return model, monitor, gen_train, gen_val, validation_size

In [0]:
# !dpkg -i rubberband-cli_1.8.1-4_amd64.deb
# !pip install jams
# !pip install soundfile
# !pip install jsonpickle
# !pip install pescador
# !pip uninstall pumpp
# !pip uninstall keras
# !pip install crema

# !pip uninstall crema

In [10]:
import pickle
import math
from tqdm import tqdm
import pandas as pd
import json
import glob
from sklearn.model_selection import ShuffleSplit

!pip install keras==2.1.2
# !pip install keras
# import keras as K


try:
    import keras as K
except ImportError:
    !pip install -q keras==2.1.2
    import keras as K

try:
    import pescador
except ImportError:
    !pip install -q pescador
    import pescador

try:
    import jams
    from jams.util import smkdirs
except ImportError:
    !pip install -q jams
    import jams
    from jams.util import smkdirs

try:
    import librosa
except ImportError:
    # !pip install -q librosa==0.6.2
    !pip install -q librosa
    import librosa

try:
    import muda
except ImportError:
    !pip install -q muda
    import muda

import pumpp
from crema import layers as custom_layers
from crema import utils as custom_utils

# checkpoint_path = os.path.join(output_path, 'checkpoint_model.h5')

smkdirs(output_path)

#version = crema_custom.utils.increment_version(os.path.join(output_path, 'version.txt'))
model, monitor, gen_train, gen_val, validation_size = prepare_model(working,
                                                max_samples, duration,
                                                rate, batch_size,
                                                epochs, epoch_size,
                                                early_stopping,
                                                reduce_lr, seed, 
                                                # checkpoint_path=os.path.join(output_path, 'saved_models/weights-improvement-ep20-acc0.66-chord_loss1.34.hdf5')
                                                                        )

# Construct the checkpoint path

checkpoint_path = os.path.join(output_path, "saved_models/weights-improvement-ep{epoch:02d}-acc{val_chord_tag_sparse_categorical_accuracy:.2f}-chord_loss{val_chord_tag_loss:.2f}.hdf5")
# checkpoint = ModelCheckpoint(filepath, monitor=monitor, verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# print("checkpoint_path is {}".format(checkpoint_path))
# Build the callbacks
cb = []

class CustomSaver(K.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("output_results/saved_models/model_{}.hd5".format(epoch+1))
saver = CustomSaver()
cb.append(saver)

cb.append(K.callbacks.ModelCheckpoint(checkpoint_path,
                                        save_best_only=True,
                                        verbose=1,
                                        monitor=monitor))

cb.append(K.callbacks.ReduceLROnPlateau(patience=reduce_lr,
                                        verbose=1,
                                        monitor=monitor))

cb.append(K.callbacks.EarlyStopping(patience=early_stopping,
                                    verbose=1,
                                    monitor=monitor))

W0903 11:13:39.145034 140517761030016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:497: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 11:13:39.155191 140517761030016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:64: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 11:13:39.208545 140517761030016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3636: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 11:13:39.669530 140517761030016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1247: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructio

In [0]:
# Fit the model
history = model.fit_generator(gen_train, epoch_size, epochs,
                    validation_data=gen_val,
                    validation_steps=validation_size,
                    callbacks=cb,
                    # initial_epoch=20
                    )

W0903 11:13:51.757548 140517761030016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:958: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0903 11:13:51.772784 140517761030016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:680: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/500
   2/2048 [..............................] - ETA: 13:56:28 - loss: 11.8369 - chord_tag_loss: 5.5940 - chord_pitch_loss: 0.7499 - chord_root_loss: 2.7720 - chord_bass_loss: 2.7210 - chord_tag_sparse_categorical_accuracy: 0.0051

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (1.145345). Check your callbacks.
  % delta_t_median)


   3/2048 [..............................] - ETA: 9:41:35 - loss: 11.9034 - chord_tag_loss: 5.6185 - chord_pitch_loss: 0.7525 - chord_root_loss: 2.7821 - chord_bass_loss: 2.7503 - chord_tag_sparse_categorical_accuracy: 0.0065 

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.958461). Check your callbacks.
  % delta_t_median)


2048/2048 [==============================] - 1301s 635ms/step - loss: 11.5554 - chord_tag_loss: 5.5823 - chord_pitch_loss: 0.7462 - chord_root_loss: 2.6354 - chord_bass_loss: 2.5915 - chord_tag_sparse_categorical_accuracy: 0.0096 - val_loss: 10.5202 - val_chord_tag_loss: 5.2464 - val_chord_pitch_loss: 0.7240 - val_chord_root_loss: 2.3049 - val_chord_bass_loss: 2.2449 - val_chord_tag_sparse_categorical_accuracy: 0.0176
Epoch 2/500
2048/2048 [==============================] - 890s 434ms/step - loss: 10.7528 - chord_tag_loss: 5.2918 - chord_pitch_loss: 0.7301 - chord_root_loss: 2.3820 - chord_bass_loss: 2.3490 - chord_tag_sparse_categorical_accuracy: 0.0793 - val_loss: 10.0338 - val_chord_tag_loss: 5.0596 - val_chord_pitch_loss: 0.7216 - val_chord_root_loss: 2.1149 - val_chord_bass_loss: 2.1376 - val_chord_tag_sparse_categorical_accuracy: 0.1974
Epoch 3/500
2048/2048 [==============================] - 898s 438ms/step - loss: 10.0503 - chord_tag_loss: 5.0718 - chord_pitch_loss: 0.7084 - ch